In [1]:
import pandas as pd
import json
import os
import deepsig
from IPython.display import display

2024-05-18 12:54:23.467630: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 12:54:23.467658: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 12:54:23.468373: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-18 12:54:23.472568: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-18 12:54:24.125837: W tensorflow/compiler/tf2

In [2]:
cols = ['dataset', 'method', 'fitness_rule', 'fitness', 'ACC', 'MCC', 'f1_score', 'avg_odds_diff', 'stat_par_diff', 'eq_opp_diff']

In [3]:
mlp_baseline_results = pd.read_csv('simple_mlp_results.csv')
mlp_baseline_results.replace({'simple_mlp_initializer': 'MLP'}, inplace=True)

mlp_pearson_results = pd.read_csv('mlp_preg_results.csv')
mlp_pearson_results.replace({'mlp_preg_initializer': 'PReg-MLP'}, inplace=True)

mlp_spearman_results = pd.read_csv('mlp_sreg_results.csv')
mlp_spearman_results.replace({'mlp_sreg_initializer': 'SReg-MLP'}, inplace=True)

#mlp_auto_reg_results = pd.read_csv('mlp_auto_reg_results.csv')
#mlp_auto_reg_results.replace({'mlp_auto_reg_initializer': 'AutoReg-MLP'}, inplace=True)

mlp_xi_reg_results = pd.read_csv('mlp_xi_reg_results.csv')
mlp_xi_reg_results.replace({'mlp_xi_reg_initializer': 'XiReg-MLP'}, inplace=True)

mlp_results = pd.concat([mlp_baseline_results, mlp_pearson_results, mlp_spearman_results, mlp_xi_reg_results])

ftl_baseline_results = pd.read_csv('ftl_mlp_results.csv')
ftl_baseline_results.replace({'ftl_mlp_initializer': 'FTL'}, inplace=True)

ftl_pearson_results = pd.read_csv('ftl_mlp_preg_results.csv')
ftl_pearson_results.replace({'ftl_mlp_preg_initializer': 'PReg-FTL'}, inplace=True)

ftl_spearman_results = pd.read_csv('ftl_mlp_sreg_results.csv')
ftl_spearman_results.replace({'ftl_mlp_sreg_initializer': 'SReg-FTL'}, inplace=True)

#ftl_auto_reg_results = pd.read_csv('ftl_mlp_auto_reg_results.csv')
#ftl_auto_reg_results.replace({'ftl_mlp_auto_reg_initializer': 'AutoReg-FTL'}, inplace=True)

ftl_xi_reg_results = pd.read_csv('ftl_mlp_xi_reg_results.csv')
ftl_xi_reg_results.replace({'ftl_mlp_xi_reg_initializer': 'XiReg-FTL'}, inplace=True)

ftl_results = pd.concat([ftl_baseline_results, ftl_pearson_results, ftl_spearman_results, ftl_xi_reg_results])

In [4]:
for results in [mlp_results,ftl_results]:
    results.replace({'adult_dataset_reader': 'Adult Income', 'compas_dataset_reader': 'Compas Recidivism', 'german_dataset_reader': 'German Credit', 'bank_dataset_reader': 'Bank Marketing'}, inplace=True)
    results.rename(columns={'avg_odds_diff': 'Equalized Odds', 'stat_par_diff': 'Statistical Parity', 'eq_opp_diff': 'Equal Opportunity', 'MCC': 'Mathew Correlation', 'ACC': 'Accuracy'}, inplace=True)

In [5]:
fitness_rules_target_metrics = {
    'mcc_parity': {'performance': 'Mathew Correlation', 'fairness': 'Statistical Parity'},
    'mcc_opportunity': {'performance': 'Mathew Correlation', 'fairness': 'Equal Opportunity'},
    'mcc_odds': {'performance': 'Mathew Correlation', 'fairness': 'Equalized Odds'},
    'acc_parity': {'performance': 'Accuracy', 'fairness': 'Statistical Parity'},
    'acc_opportunity': {'performance': 'Accuracy', 'fairness': 'Equal Opportunity'},
    'acc_odds': {'performance': 'Accuracy', 'fairness': 'Equalized Odds'}
}

fitness_rules_target_metrics = {
    'mcc_parity': ('Mathew Correlation', 'Statistical Parity'),
    'mcc_opportunity': ('Mathew Correlation', 'Equal Opportunity'),
    'mcc_odds': ('Mathew Correlation', 'Equalized Odds'),
    'acc_parity': ('Accuracy', 'Statistical Parity'),
    'acc_opportunity': ('Accuracy', 'Equal Opportunity'),
    'acc_odds': ('Accuracy', 'Equalized Odds')
}
fitness_rules_abvr = {
    'mcc_parity': 'Max(MCC - Stat. Parity)',
    'mcc_opportunity': 'Max(MCC - Eq. Odds)',
    'mcc_odds': 'Max(MCC - Eq. Opp.)',
    'acc_parity': 'Max(Acc - Stat. Parity)',
    'acc_opportunity': 'Max(Acc - Eq. Odds)',
    'acc_odds': 'Max(Acc - Eq. Opp.)'
}

for results in [mlp_results,ftl_results]:
    results['Performance'] = 0
    results['Fairness'] = 0
    results['Fitness Rule'] = ''
    for fitness_rule, (performance_metric, fairness_metric) in fitness_rules_target_metrics.items():
        results.loc[results.fitness_rule == fitness_rule,'Performance'] = results.loc[results.fitness_rule == fitness_rule,performance_metric]
        results.loc[results.fitness_rule == fitness_rule,'Fairness'] = results.loc[results.fitness_rule == fitness_rule,fairness_metric]
        results.loc[results.fitness_rule == fitness_rule,'Fitness Rule Abvr'] = fitness_rules_abvr[fitness_rule]
        results.loc[results.fitness_rule == fitness_rule,'Fitness Rule'] = 'Max(%s - %s)' % fitness_rules_target_metrics[fitness_rule]

/tmp/ipykernel_2478217/3607436483.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.58517004 0.58512823 0.5840247  0.5836371  0.57625737 0.5769683
 0.58075151 0.57811729 0.57147862 0.55847176 0.56402881 0.58191431
 0.58343253 0.57740222 0.51579307 0.30958237 0.37962947 0.55899189
 0.53558872 0.2598879  0.57717035 0.50086739 0.33275221 0.37929293
 0.47729109 0.29196869 0.29037704 0.52303609 0.27480633 0.38668781
 0.50626951 0.29960979 0.4042848  0.52119218 0.2936422  0.33522388
 0.49730721 0.25112662 0.35119067 0.510052   0.26843855 0.36081008
 0.52157495 0.30114722 0.42995147 0.48267704 0.29056691 0.23139881
 0.54223416 0.23653437 0.27041017 0.52121263 0.25759348 0.23912165
 0.5223892  0.27406652 0.22303564 0.54094974 0.27699558 0.30939251
 0.51731345 0.30339828 0.37825089 0.52113734 0.26323857 0.33963196
 0.5538225  0.27850228 0.30234984 0.51877126 0.29494101 0.2648939
 0.52742877 0.24492927 0.13407928 0.535

In [6]:
datasets = ['Adult Income', 'Bank Marketing', 'Compas Recidivism','German Credit']
datasets

['Adult Income', 'Bank Marketing', 'Compas Recidivism', 'German Credit']

In [7]:
fitness_rules = ['mcc_parity', 'mcc_opportunity', 'mcc_odds', 'acc_parity', 'acc_opportunity', 'acc_odds']
fitness_rules

['mcc_parity',
 'mcc_opportunity',
 'mcc_odds',
 'acc_parity',
 'acc_opportunity',
 'acc_odds']

In [8]:
ftl_methods = ['FTL', 'PReg-FTL', 'SReg-FTL', 'XiReg-FTL']
mlp_methods = ['MLP', 'PReg-MLP', 'SReg-MLP', 'XiReg-MLP']
significances = []
grouped_results_list = []

In [9]:
for path, methods, results in zip(['mlp_multi_aso_data_list.json', 'ftl_multi_aso_data_list.json'],
                                  [mlp_methods, ftl_methods],
                                  (mlp_results,ftl_results)):
    if os.path.exists(path):
        with open(path) as file:
            multi_aso_data_list = json.load(file)
    else:    
        multi_aso_data_list = []
        for d in datasets:
            multi_aso_data = []
            for f in fitness_rules:
                methods_results = []
                for m in methods:
                    r = results.loc[ (results['dataset'] == d) &
                                         (results['fitness_rule'] == f) &
                                         (results['method'] == m) ]\
                                .fitness.tolist()
                    if len(r) == 0:
                        r = [-1]
                    methods_results.append(r)
                min_eps = deepsig.multi_aso(methods_results, confidence_level=0.95)
                multi_aso_data_list.append({'fitness_rule': f, 'dataset': d, 'min_eps': min_eps.tolist()})
        with open(path, 'w') as file:
            json.dump(multi_aso_data_list, file)
    
    aso_df_resume = []
    for aso_result in sorted(multi_aso_data_list, key=lambda x: x['dataset']):
        fitness_rule = aso_result['fitness_rule']
        dataset = aso_result['dataset']
    
        aso_df = pd.DataFrame(aso_result['min_eps'], columns=methods)
        aso_df['method'] = methods
        aso_df['dataset'] = dataset
        aso_df['fitness_rule'] = fitness_rule
        aso_df_resume.append(aso_df)
    
    print('Significance Testing')
    significance = pd.concat(aso_df_resume)
    significance.replace(fitness_rules_abvr, inplace=True)
    significance = significance.set_index(['fitness_rule', 'dataset'])
    significance = significance.sort_values(by=['fitness_rule', 'dataset'], ascending=[False, True])
    significances.append(significance)
    
    grouped_results = results\
        .groupby(['Fitness Rule Abvr', 'dataset', 'method'])\
        .agg({'fitness': ['mean', 'std', 'count'], 'Performance': ['mean', 'std'], 'Fairness': ['mean', 'std']})\
        .sort_values(by=['Fitness Rule Abvr', 'dataset', ('fitness','mean')], ascending=False)
    grouped_results['formatted_fitness'] = grouped_results.apply(lambda row: f"${row[('fitness', 'mean')]:.3f} (\pm{row[('fitness', 'std')]:.2f})$", axis=1)
    grouped_results['formatted_performance'] = grouped_results.apply(lambda row: f"${row[('Performance', 'mean')]:.3f} (\pm{row[('Performance', 'std')]:.2f})$", axis=1)
    grouped_results['formatted_fairness'] = grouped_results.apply(lambda row: f"${row[('Fairness', 'mean')]:.3f} (\pm{row[('Fairness', 'std')]:.2f})$", axis=1)
    grouped_results = grouped_results.sort_values(by=['Fitness Rule Abvr', 'dataset'])
    grouped_results_list.append(grouped_results)

Model comparisons: 100%|█████████▉| 5994/6000 [00:14<00:00, 423.38it/s]


Significance Testing


Model comparisons: 100%|█████████▉| 5994/6000 [00:13<00:00, 442.12it/s]

Significance Testing


In [10]:
significances[0].sort_values(by=['fitness_rule', 'dataset'], ascending=[False, True])

MLP  PReg-MLP  SReg-MLP  \
fitness_rule            dataset                                           
Max(MCC - Stat. Parity) Adult Income       1.000000  1.000000  0.687397   
                        Adult Income       0.236552  1.000000  0.146357   
                        Adult Income       1.000000  1.000000  1.000000   
                        Adult Income       0.825914  1.000000  0.618027   
                        Bank Marketing     1.000000  0.550131  0.476932   
...                                             ...       ...       ...   
Max(Acc - Eq. Odds)     Compas Recidivism  0.054865  0.283360  0.354095   
                        German Credit      1.000000  1.000000  1.000000   
                        German Credit      0.425855  1.000000  0.388310   
                        German Credit      1.000000  1.000000  1.000000   
                        German Credit      0.270305  0.518537  0.228995   

                                           XiReg-MLP     method  
fitness_rule            dataset                                  
Max(MCC - Stat. Parity) Adult Income         1.00000        MLP  
                        Adult Income         0.32503   PReg-MLP  
                        Adult Income         1.00000   SReg-MLP  
                        Adult Income         1.00000  XiReg-MLP  
                        Bank Marketing       1.00000        MLP  
...                                              ...        ...  
Max(Acc - Eq. Odds)     Compas Recidivism    1.00000  XiReg-MLP  
                        German Credit        1.00000        MLP  
                        German Credit        1.00000   PReg-MLP  
                        German Credit        1.00000   SReg-MLP  
                        German Credit        1.00000  XiReg-MLP  

[96 rows x 5 columns]

In [11]:
grouped_results_list[0]

fitness                  \
                                                         mean       std count   
Fitness Rule Abvr       dataset           method                                
Max(Acc - Eq. Odds)     Adult Income      MLP        0.758290  0.035161    15   
                                          XiReg-MLP  0.757171  0.050661    15   
                                          PReg-MLP   0.749909  0.036699    16   
                                          SReg-MLP   0.747867  0.032609    16   
                        Bank Marketing    XiReg-MLP  0.839229  0.045837    15   
...                                                       ...       ...   ...   
Max(MCC - Stat. Parity) Compas Recidivism MLP        0.074028  0.034694    30   
                        German Credit     MLP        0.265661  0.099801    30   
                                          PReg-MLP   0.254748  0.071677    16   
                                          XiReg-MLP  0.245571  0.070406    15   
                                          SReg-MLP   0.192318  0.107623    16   

                                                    Performance            \
                                                           mean       std   
Fitness Rule Abvr       dataset           method                            
Max(Acc - Eq. Odds)     Adult Income      MLP          0.847864  0.004669   
                                          XiReg-MLP    0.848624  0.002018   
                                          PReg-MLP     0.848183  0.004362   
                                          SReg-MLP     0.848653  0.003668   
                        Bank Marketing    XiReg-MLP    0.902514  0.004113   
...                                                         ...       ...   
Max(MCC - Stat. Parity) Compas Recidivism MLP          0.283302  0.021736   
                        German Credit     MLP          0.329468  0.090834   
                                          PReg-MLP     0.341448  0.067708   
                                          XiReg-MLP    0.329368  0.050637   
                                          SReg-MLP     0.291751  0.073053   

                                                     Fairness            \
                                                         mean       std   
Fitness Rule Abvr       dataset           method                          
Max(Acc - Eq. Odds)     Adult Income      MLP        0.089574  0.035575   
                                          XiReg-MLP  0.091452  0.051310   
                                          PReg-MLP   0.098273  0.039655   
                                          SReg-MLP   0.100786  0.031567   
                        Bank Marketing    XiReg-MLP  0.063285  0.044914   
...                                                       ...       ...   
Max(MCC - Stat. Parity) Compas Recidivism MLP        0.209274  0.038248   
                        German Credit     MLP        0.063807  0.046639   
                                          PReg-MLP   0.086701  0.042336   
                                          XiReg-MLP  0.083796  0.060963   
                                          SReg-MLP   0.099432  0.060881   

                                                     formatted_fitness  \
                                                                         
Fitness Rule Abvr       dataset           method                         
Max(Acc - Eq. Odds)     Adult Income      MLP        $0.758 (\pm0.04)$   
                                          XiReg-MLP  $0.757 (\pm0.05)$   
                                          PReg-MLP   $0.750 (\pm0.04)$   
                                          SReg-MLP   $0.748 (\pm0.03)$   
                        Bank Marketing    XiReg-MLP  $0.839 (\pm0.05)$   
...                                                                ...   
Max(MCC - Stat. Parity) Compas Recidivism MLP        $0.074 (\pm0.03)$   
                        German Credit     MLP    

In [12]:
significances[1].sort_values(by=['fitness_rule', 'dataset'], ascending=[False, True])

FTL  PReg-FTL  SReg-FTL  \
fitness_rule            dataset                                           
Max(MCC - Stat. Parity) Adult Income       1.000000  1.000000  0.383339   
                        Adult Income       0.463432  1.000000  0.210242   
                        Adult Income       1.000000  1.000000  1.000000   
                        Adult Income       0.349166  0.881989  0.114389   
                        Bank Marketing     1.000000  0.697897  1.000000   
...                                             ...       ...       ...   
Max(Acc - Eq. Odds)     Compas Recidivism  1.000000  1.000000  1.000000   
                        German Credit      1.000000  1.000000  0.724423   
                        German Credit      0.267596  1.000000  0.085384   
                        German Credit      1.000000  1.000000  1.000000   
                        German Credit      1.000000  1.000000  0.978734   

                                           XiReg-FTL     method  
fitness_rule            dataset                                  
Max(MCC - Stat. Parity) Adult Income        1.000000        FTL  
                        Adult Income        1.000000   PReg-FTL  
                        Adult Income        1.000000   SReg-FTL  
                        Adult Income        1.000000  XiReg-FTL  
                        Bank Marketing      0.481308        FTL  
...                                              ...        ...  
Max(Acc - Eq. Odds)     Compas Recidivism   1.000000  XiReg-FTL  
                        German Credit       0.765238        FTL  
                        German Credit       0.134237   PReg-FTL  
                        German Credit       1.000000   SReg-FTL  
                        German Credit       1.000000  XiReg-FTL  

[96 rows x 5 columns]

In [13]:
grouped_results_list[1]

fitness                  \
                                                         mean       std count   
Fitness Rule Abvr       dataset           method                                
Max(Acc - Eq. Odds)     Adult Income      PReg-FTL   0.816948  0.023932    17   
                                          XiReg-FTL  0.815293  0.020755    15   
                                          FTL        0.811852  0.025179    24   
                                          SReg-FTL   0.796648  0.036947    13   
                        Bank Marketing    SReg-FTL   0.805073  0.086488    15   
...                                                       ...       ...   ...   
Max(MCC - Stat. Parity) Compas Recidivism PReg-FTL   0.221342  0.113259    15   
                        German Credit     XiReg-FTL  0.302067  0.062075    15   
                                          FTL        0.255934  0.122275    14   
                                          PReg-FTL   0.249641  0.078402    14   
                                          SReg-FTL   0.245569  0.106336    14   

                                                    Performance            \
                                                           mean       std   
Fitness Rule Abvr       dataset           method                            
Max(Acc - Eq. Odds)     Adult Income      PReg-FTL     0.844002  0.008850   
                                          XiReg-FTL    0.846781  0.004568   
                                          FTL          0.845393  0.005262   
                                          SReg-FTL     0.843398  0.003089   
                        Bank Marketing    SReg-FTL     0.898295  0.005811   
...                                                         ...       ...   
Max(MCC - Stat. Parity) Compas Recidivism PReg-FTL     0.277947  0.036233   
                        German Credit     XiReg-FTL    0.370891  0.053377   
                                          FTL          0.354889  0.077343   
                                          PReg-FTL     0.336466  0.041280   
                                          SReg-FTL     0.326044  0.080928   

                                                     Fairness            \
                                                         mean       std   
Fitness Rule Abvr       dataset           method                          
Max(Acc - Eq. Odds)     Adult Income      PReg-FTL   0.027055  0.022190   
                                          XiReg-FTL  0.031488  0.021377   
                                          FTL        0.033541  0.023934   
                                          SReg-FTL   0.046751  0.037872   
                        Bank Marketing    SReg-FTL   0.093222  0.087795   
...                                                       ...       ...   
Max(MCC - Stat. Parity) Compas Recidivism PReg-FTL   0.056605  0.085058   
                        German Credit     XiReg-FTL  0.068824  0.049439   
                                          FTL        0.098955  0.062630   
                                          PReg-FTL   0.086824  0.056425   
                                          SReg-FTL   0.080474  0.054080   

                                                     formatted_fitness  \
                                                                         
Fitness Rule Abvr       dataset           method                         
Max(Acc - Eq. Odds)     Adult Income      PReg-FTL   $0.817 (\pm0.02)$   
                                          XiReg-FTL  $0.815 (\pm0.02)$   
                                          FTL        $0.812 (\pm0.03)$   
                                          SReg-FTL   $0.797 (\pm0.04)$   
                        Bank Marketing    SReg-FTL   $0.805 (\pm0.09)$   
...                                                                ...   
Max(MCC - Stat. Parity) Compas Recidivism PReg-FTL   $0.221 (\pm0.11)$   
                        German Credit     XiReg-F